In [1]:
import requests, bs4
import pandas as pd
from urllib.parse import urlencode, quote_plus, unquote
from keys import worknetKey

url = 'http://openapi.work.go.kr/opi/opi/opia/wantedApi.do'
authKey = unquote(worknetKey)

In [2]:
queryParams = '?' + urlencode(
    {
        quote_plus('authKey') : authKey,
        quote_plus('callTp'): 'L',
        quote_plus('returnType'): 'XML',
        quote_plus('startPage'): '1',
        quote_plus('display'): '100',
        quote_plus('pfPreferential'): 'B', # 홈페이지 명세에는 S로 되어있는데, B가 시니어 공고임
        # quote_plus('region'): '45110' # 전라북도 전주시 코드.
     }
)

response = requests.get(url + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
rows = xmlobj.findAll('wanted')

In [3]:
# 모든 행과 열의 값을 모아 매트릭스로 만들어보자.
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장 -> maxEdubg는 있는 데이터도 있고, 없는 데이터도 있어서 그냥 안받도록 함. busino는 거의 없는 것 같으나, 혹시 몰라 안받도록 함.
        if i == 0 and columns[j].name != 'maxEdubg' and columns[j].name != 'busino':
            nameList.append(columns[j].name)
        # 컬럼 값은 모든 행의 값을 저장
        if columns[j].name != 'maxEdubg' and columns[j].name != 'busino':
            eachColumn = columns[j].text
            columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []  # 다음 row의 값을 넣기 위해 비워준다.

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
result = pd.DataFrame(rowList, columns=nameList)
result

,wantedAuthNo,company,title,salTpNm,sal,minSal,maxSal,region,holidayTpNm,minEdubg,career,regDt,closeDt,infoSvc,wantedInfoUrl,wantedMobileInfoUrl,smodifyDtm,zipCd,strtnmCd,basicAddr,detailAddr,empTpCd,jobsCd,prefCd
0,KJLD002107130005,진영산업(주),안전관리자 채용,월급,250만원 ~ 300만원,2500000,3000000,강원 동해시,주5일근무,학력무관,관계없음,21-07-13,채용시까지 21-09-12,VALIDATION,http://www.work.go.kr/empDetailRedirect.do?wan...,https://m.work.go.kr/regionJobsWorknet/jobDeta...,202107131920,25779,421703221014,강원도 동해시 대동로 210,쌍용 씨앤이(주) 북평공장 내,10,140206,
1,K151152107130178,주식회사청명엔텍,"N,C,T 기계 가공 기사 ,프로그램머 운영기사...",연봉,3000만원 ~ 4000만원,30000000,40000000,경기 화성시,주5일근무,고졸,경력,21-07-13,21-08-20,VALIDATION,http://www.work.go.kr/empDetailRedirect.do?wan...,https://m.work.go.kr/regionJobsWorknet/jobDeta...,202107131830,18364,415903210075,경기도 화성시 안녕남로 57-1,,10,159103,
2,K120142107130040,주식회사 휴먼에스엔씨,[급구] 환경미화(주간 2명),시급,9020원,9020,0,충북 청주시 흥덕구,주5일근무,학력무관,경력,21-07-13,21-07-31,VALIDATION,http://www.work.go.kr/empDetailRedirect.do?wan...,https://m.work.go.kr/regionJobsWorknet/jobDeta...,202107131801,28101,431133240053,충청북도 청주시 흥덕구 옥산면 오산가좌로 415-14,,20,561101,
3,KJKR002107130030,인덕종합관리(주),오남읍 아파트 전기과장 채용,월급,316만원,3160000,0,경기 남양주시,주5일근무,고졸,경력,21-07-13,21-09-12,VALIDATION,http://www.work.go.kr/empDetailRedirect.do?wan...,https://m.work.go.kr/regionJobsWorknet/jobDeta...,202107131756,12042,413603197050,경기도 남양주시 오남읍 진건오남로 735-10,.,10,834001,
4,K172112107130084,성백산업,식품(음료) 포장 직원 모집,월급,200만원 ~ 250만원,2000000,2500000,충남 천안시 서북구,주5일근무,학력무관,관계없음,21-07-13,21-07-27,VALIDATION,http://www.work.go.kr/empDetailRedirect.do?wan...,https://m.work.go.kr/regionJobsWorknet/jobDeta...,202107131755,31075,441333250002,충청남도 천안시 서북구 2공단2로 112,성백산업,10,890006,
5,KF10152107130005,아리아케어,재가요양보호사 모집,시급,13000원 ~ 13000원,13000,13000,인천 남동구,주5일근무,학력무관,신입,21-07-13,채용시까지 21-09-06,VALIDATION,http://www.work.go.kr/empDetailRedirect.do?wan...,https://m.work.go.kr/regionJobsWorknet/jobDeta...,202107131754,21535,282004259109,인천광역시 남동구 구월말로58번길 15-15,만수동,11,550103,
6,KJKL002107130019,폴리텍 주식회사,아파트 청소원 모집,시급,8720원,8720,0,경기 부천시,주6일근무,학력무관,경력,21-07-13,채용시까지 21-09-12,VALIDATION,http://www.work.go.kr/empDetailRedirect.do?wan...,https://m.work.go.kr/regionJobsWorknet/jobDeta...,202107131748,14731,411902000015,경기도 부천시 송내대로 14,관리사무실,20,561101,
7,KJKE002107130022,정은시니어재가센터,재가요양보호사 모집,시급,10850원,10850,0,경기 광명시,주5일근무,학력무관,관계없음,21-07-13,21-09-12,VALIDATION,http://www.work.go.kr/empDetailRedirect.do?wan...,https://m.work.go.kr/regionJobsWorknet/jobDeta...,202107131743,14287,412103000122,경기도 광명시 광명로 877,상세주소미기입,11,550104,
8,KF10132107130009,양지재가장기요양기관,재가요양보호사(주안8동),시급,11100원 ~ 11100원,11100,11100,인천 미추홀구,주5일근무,학력무관,신입,21-07-13,채용시까지 21-09-12,VALIDATION,http://www.work.go.kr/empDetailRedirect.do?wan...,https://m.work.go.kr/regionJobsWorknet/jobDeta...,202107131734,22227,281774253259,인천광역시 미추홀구 미추홀대로578번길 13-1,신기시장근처,11,550104,
9,K141122107130046,휴먼뱅크(주),골프장 코스관리원 모집,시급,8720원,8720,0,경북 경주시,주6일근무,학력무관,관계없음,21-07-13,채용시까지 21-07-31,VALIDATION,http://www.work.go.kr/empDetailRedirect.do?wan...,https://m.work.go.kr/regionJobsWorknet/jobDeta...,202107131733,38116,471303305021,경상북도 경주시 보문로 319,신라컨트리클럽,20,562900,


In [4]:
wantedAuthNo = []
rowsLen = len(rows)
for i in range(0, rowsLen):
  new = rowList[i]
  wantedAuthNo.append(new[0])

rows_detail = []
for i in range(0, rowsLen):
  queryParams_detail = '?' + urlencode(
      {
          quote_plus('authKey') : authKey,
          quote_plus('callTp'): 'D',
          quote_plus('returnType'): 'XML',
          quote_plus('wantedAuthNo'): wantedAuthNo[i],
          quote_plus('infoSvc'): 'VALIDATION'
      }
  )

  response_detail = requests.get(url + queryParams_detail).text.encode('utf-8')
  xmlobj_detail = bs4.BeautifulSoup(response_detail, 'lxml-xml')
  rows_detail.append(xmlobj_detail.findAll('wantedInfo') + xmlobj_detail.findAll('empchargeInfo'))

In [5]:
rowList_detail = []
nameList_detail = []
columnList_detail = []

rowsLen = len(rows_detail)
for i in range(0, rowsLen):
    row = rows_detail[i]
    columns = row[0].find_all() + row[1].find_all()
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        name = columns[j].name
        if i == 0 and name != 'attachFileInfo' and name != 'corpAttachList' and name != 'keywordList' and name != 'attachFileUrl' and name != 'srchKeywordNm':
            nameList_detail.append(columns[j].name)
        if name != 'attachFileInfo' and name != 'corpAttachList' and name != 'keywordList' and name != 'attachFileUrl' and name != 'srchKeywordNm':
            eachColumn = columns[j].text
            columnList_detail.append(eachColumn)
    rowList_detail.append(columnList_detail)
    columnList_detail = []  # 다음 row의 값을 넣기 위해 비워준다.

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
result_detail = pd.DataFrame(rowList_detail, columns=nameList_detail)
result_detail

,jobsNm,wantedTitle,relJobsNm,jobCont,receiptCloseDt,empTpNm,collectPsncnt,salTpNm,enterTpNm,eduNm,forLang,major,certificate,mltsvcExcHope,compAbl,pfCond,etcPfCond,selMthd,rcptMthd,submitDoc,etcHopeCont,workRegion,nearLine,workdayWorkhrCont,fourIns,retirepay,etcWelfare,disableCvntl,dtlRecrContUrl,jobsCd,minEdubgIcd,maxEdubgIcd,regionCd,empTpCd,enterTpCd,salTpCd,staAreaRegionCd,lineCd,staNmCd,exitNoCd,walkDistCd,empChargerDpt,contactTelno,chargerFaxNo
0,건축안전·환경·품질·에너지관리 기술자(140206),안전관리자 채용,,현장안전관리 및 간단한 사무,채용시까지,기간의 정함이 없는 근로계약/ 파견근로 비희망,1,"월급2,500,000원 이상 ~ 3,000,000원 이하, 면접 후 재조정 가능",관계없음,학력무관,,,,비희망,문서작성 (워드프로세스 활용),"(준)고령자(50세이상), 운전면허증, 차량소지자",* 시멘트 회사 근무경험자 (우대),"서류,면접",방문,이력서,,"(25779) 강원도 동해시 대동로 210, 쌍용 씨앤이(주) 북평공장 내 (송정동)",,"평일 : (오전) 8시 00분 ~ (오후) 5시 00분, 주 5일 근무, 평균근무시...",국민연금 고용보험 산재보험 의료보험,퇴직금,기타(* 1식 이외 업무 시간대에 따라 식사 별도 제공),,http://www.work.go.kr/outOffer/empInfo/empDeta...,140206,00,00,42170,10,Z,M,,,,,,,033-520-5076,033-521-6139
1,기계·금속 제도사(캐드원)(159103),"N,C,T 기계 가공 기사 ,프로그램머 운영기사 구인",전기 부품·제품 생산기계 조작원 금속 공작기계 조작원,"당사는 신발건조기 및 우산건조기 전문제조회사 이며 , 모든 제품은 특허 제품이므로 ...",20210820,기간의 정함이 없는 근로계약/ 파견근로 비희망,4,"연봉30,000,000원 이상 ~ 40,000,000원 이하, 상여금 : 100(미...",경력 (최소1년) 필수,고졸-대졸(2~3년),,,자동차운전면허,비희망,문서작성 (워드프로세스 활용),"(준)고령자(50세이상), 운전면허증, 차량소지자",,"서류,면접","방문,팩스,이메일","이력서,자기소개서",,(18364) 경기도 화성시 안녕남로 57-1 (안녕동),1호선 병점 4번출구 3KM,"평일 : (오전) 8시 30분~(오후) 5시 30분, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,직원대출 제도,,http://www.work.go.kr/outOffer/empInfo/empDeta...,159103,03,04,41590,10,E,Y,11000,1,34,4,11,,031-222-8893,031-222-8895
2,"건물 청소원(공공건물,아파트,사무실,병원,상가,공장 등)(561101)",[급구] 환경미화(주간 2명),,주6일 주간근무 - 2명 \r\n주간 근무 (08:30~17:30)\r\n- 휴일근...,20210731,기간의 정함이 있는 근로계약12 개월/ 파견근로 비희망,2,"시급9,020원 이상,",경력 (최소1년) 필수,학력무관,,,,비희망,,(준)고령자(50세이상),,"서류,면접","방문,팩스,워크넷,이메일",이력서,,(28101) 충청북도 청주시 흥덕구 옥산면 오산가좌로 415-14,,"평일 : 주간근무) 08:30~17:30, 주 5일 근무",국민연금 고용보험 산재보험 의료보험,퇴직연금,통근버스,,http://www.work.go.kr/outOffer/empInfo/empDeta...,561101,00,00,43113,20,E,H,,,,,,,02-2057-2126,02-2057-2125
3,아파트 전기관리원(834001),오남읍 아파트 전기과장 채용,,"- 아파트 기전실 관리 업무및 인원관리 외\r\n- 전기산업기사이상 (1,350 K...",20210912,기간의 정함이 없는 근로계약/ 파견근로 비희망,1,"월급3,160,000원 이상,",경력 (최소2년) 필수,고졸-박사,,,,비희망,,(준)고령자(50세이상),영선 업무 가능자 우대,"서류,면접","팩스,이메일,기타(- 이력서 팩스 지원( 02-983-0094, 상단에 &quot;...",이력서,전화문의 사절\r\n기전실 경력2년이상자,"(12042) 경기도 남양주시 오남읍 진건오남로 735-10, . (한국아파트)",,"평일 : (오전) 9시 00분 ~ (오후) 6시 00분, 주 5일 근무, 평균근무시...",국민연금 고용보험 산재보험 의료보험,퇴직금,기타(하계휴가비),,http://www.work.go.kr/outOffer/empInfo/empDeta...,834001,03,07,41360,10,E,M,,,,,,,02-467-6500,02-983-0094
4,식품 분야 단순 종사원(890006),식품(음료) 포장 직원 모집,음료 제조기계 조작원,1. 제품 좌우 정열\r\n2. 박스내 간지투입\r\n3. 용기 투입,20210727,기간의 정함이 없는 근로계약/ 파견근로 비희망,2,"월급2,000,000원 이상 ~ 2,500,000원 이하, 상여금 : 200(미 포...",관계없음,학력무관,,,,비희망,,"(준)고령자(50세이상), 차량소지자",,"면접,기타(1.워크넷 이력서 참조\r\n2.전화 연락(010-6772-4760))","방문,워크넷,이메일,기타(1.워크넷 이력서 참조\r\n2.전화 연락(010-6772...","이력서,기타(이력서)",본 구인관련 초과근무 및 수습기간 여부는 지원시 한번 더 확인하시기 바랍니다.,"(31075) 충청남도 천안시 서북구 2공단2로 112, 성백산업 (업성동)",1호선 두정 1번출구 3KM,"평일 : 07:00~15:00/15:00~23:00, 주 5일 근무, 평균근무시간 ...",국민연금 고용보험 산재보험 의료보험,퇴직금,통근버스,,http://www.work.go.kr/outOffer/empInfo/empDeta...,890006,00,00,44133,10,Z,M,11000,1,29,1,11,,041-559-7246,041-563-4760
5,시설 요양보호사(노인요양사)(550103),재가요양보호사 모집,,- 모래내시장 근처 치매5등급 할아버지 재가요양보호사 구인\r\n- 09:00~12...,채용시까지,기간의 정함이 없는 근로계약(시간(선택)제)/ 파견근로 비희망,1,"시급13,000원 이상 ~ 13,000원 이하,",신입,학력무관,,,요양보호사,비희망,,(준)고령자(50세이상),,면접,"방문,기타(전화 후 방문)","기타(자격증, 사진, 건강검진서)",,"(21535) 인천광역시 남동구 구월말로58번길 15-15, 만수동 (만수동, 우...",,"평일 : (오전) 9시 00분 ~ (정오) 12시 00분, 주 5일 근무",국민연금 고용보험 산재보험 의료보험,퇴직금,,,http://www.work.go.kr/outOffer/empInfo/empDeta...,550103,00,00,28200,11,N,H,,,,,,,032-504-9979,032-504-9980
6,"건물 청소원(공공건물,아파트,사무실,병원,상가,공장 등)(561101)",아파트 청소원 모집,,- 송내파인푸르지오 아파트 내부청소\r\n- 근무시간:주6일(평일)09:00~16:...,채용시까지,기간의 정함이 있는 근로계약12 개월/ 파견근로 비희망,1,"시급8,720원 이상,",경력 (최소1년) 필수,학력무관,,,,비희망,,(준)고령자(50세이상),,"서류,면접","방문,기타(통화 후 방문면접)",이력서,,"(14731) 경기도 부천시 송내대로 14, 관리사무실 (송내동, 송내역파인푸르지...",5호선 애오개 2번출구 200M,"평일 : (평일)09:00~16:00\r\n(토요일)09:00~12:00, 주 6일 근무"